# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
# 1) Seu código aqui

url = "http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado"

In [5]:
r = requests.get(url)

In [6]:
r.status_code

200

In [9]:
data_json = r.json()

In [11]:
#Apenas para observar se os dados estão corretos 
#data_json["items"]

In [104]:
# 2) Seu código aqui
lista = pd.DataFrame(data_json["items"])

In [25]:
lista.dtypes

id_pleito                  int64
tipo_interessado          object
interessado               object
cod_ibge                   int64
uf                        object
num_pvl                   object
status                    object
num_processo              object
data_protocolo            object
tipo_operacao             object
finalidade                object
tipo_credor               object
credor                    object
moeda                     object
valor                    float64
pvl_assoc_divida           int64
pvl_contradado_credor      int64
data_status               object
dtype: object

In [19]:
lista.status.explode().value_counts().head(3)

status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: count, dtype: int64

In [21]:
lista.status.explode().value_counts()

status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: count, dtype: int64

In [ ]:
# 3) Seu código aqui
#Construa uma nova variável que contenha o ano do status. Observe que data_status vem como tipo object no DataFrame. 
#Dica: você pode usar o método .str para transformar o tipo da variável em string, em seguida um método como slice() ou split().


In [83]:
ano = lista["data_status"].str.split("/", expand= True)[2]
ano.head(3)

0    2009
1    2013
2    2013
Name: 2, dtype: object

In [85]:
# 4) 
#Indique a frequência de cada ano do campo construído no item (3).

ano.value_counts()

2
2012    11
2009     9
2013     7
2011     6
2002     4
2016     4
2014     4
2010     3
2008     2
2006     2
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [101]:
#1) Seu código aqui
#Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado ('Estado'ou Município), e que devolve os dados da consulta no formato DataFrame.

In [112]:
def comparar_uf(uf: str, tipo_interessado: str):
    r = requests.get("https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl")
    data = r.json()
    df = pd.DataFrame(data["items"])

    #Filtrando a UF e retorno
    filtro = (df["uf"] == uf) & (df["tipo_interessado"] == tipo_interessado)
    df_filtrada = df[filtro]

    return df_filtrada

In [97]:
# 2) Seu código aqui
#Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 'Arquivado por decurso de prazo' estão registradas?bb

In [120]:
#teste

#uf = "SP"
#tipo_interessado = "Estado"
#df_filtrado = comparar_uf(uf, tipo_interessado)
#df_filtrado.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
158,18661,Estado,São Paulo,35,SP,00000.000000/1003-61,Deferido,17944.000345/2015-09,2017-04-24T11:46:59Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,4.759899e+08,1,0,10/05/2017
211,7947,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000682/2014-15,2014-06-11T00:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,4.801350e+08,1,0,15/08/2014
758,5248,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000068/2013-64,2014-07-01T00:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,2.000000e+08,1,0,07/08/2014


In [116]:
uf = "MG"
tipo_interessado = "Estado"

df_filtrado = comparar_uf(uf, tipo_interessado)

df_arquivada = df_filtrado[df_filtrado["status"] == "Arquivado por decurso de prazo"]
df_arquivada

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status


In [93]:
# 3) Seu código aqui 
#Qual é o município da Bahia com mais solicitações deferidas?

In [182]:
uf = "BA"
tipo_interessado = "Município"
df_filtrado = comparar_uf(uf, tipo_interessado)
municipio_ba = df_filtrado
municipio_ba.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
35,27353,Município,Amargosa,2901007,BA,PVL02.002662/2017-51,Arquivado por decurso de prazo,17944.100262/2018-53,2017-12-29T11:25:41Z,Operação contratual interna,Multissetorial,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4350000.0,0,0,14/08/2018
60,31372,Município,Santa Brígida,2927606,BA,PVL02.000521/2019-65,Em retificação pelo interessado,17944.101204/2019-28,2019-04-05T11:09:11Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Companhia de Eletricidade do Estado da Bahia,Real,48721.3,0,0,11/04/2019
93,6400,Município,Barra do Mendes,2903003,BA,None,Arquivado por decurso de prazo,17944.000337/2009-14,2009-03-30T00:00:00Z,Operação contratual interna,Provias,Instituição Financeira Nacional,Banco do Brasil S/A,Real,400000.0,0,0,07/04/2009


In [184]:
status = municipio_ba[municipio_ba["status"] == "Deferido"]
status.interessado.value_counts()

interessado
Luís Eduardo Magalhães    4
Vitória da Conquista      3
Formosa do Rio Preto      2
Campo Formoso             2
Candeias                  2
                         ..
Amélia Rodrigues          1
Irará                     1
Serra Preta               1
Macarani                  1
Coribe                    1
Name: count, Length: 69, dtype: int64

In [95]:
# 4) Seu código aqui
#Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [190]:
uf = "BA"
tipo_interessado = "Estado"
df_filtrado = comparar_uf(uf, tipo_interessado)
uf_ba = df_filtrado

uf_ba.to_csv("Solicitações_Bahia")